#Super Model (Scenario 3a, 6a)

Super Model (Single model made with combined training sets)

*   The attacker collects training data X1 to X7.
*   He combines them to form a large training set.
*   On that large training dataset, he performs the 10 fold cross validation.

Before running the following cell upload all fixed stressor files where p is either 1,2 or 4 (The combined dataset should have 1400 datapoints)

In [1]:
import json
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report

In [2]:
#Known-Noise (Adversarial Training)
values = {}
def classify_ten_fold_known_noise(Xdata, ydata):
    skf = StratifiedShuffleSplit(n_splits=10, random_state=42)
    partial_accuracies=[]
    for train_index, test_index in skf.split(Xdata, ydata):
        X_train, X_test = Xdata[train_index], Xdata[test_index]
        mean=X_train.mean()
        std=X_train.std()
        X_train_norm=(X_train-mean+1e-10)/std
        X_test_norm=(X_test-mean+1e-10)/std

        y_train, y_test = ydata[train_index], ydata[test_index]


        model=LogisticRegression(max_iter=1000)
        model.fit(X_train_norm,y_train)
        prd=model.predict(X_test_norm)

        partial_accuracies.append(classification_report(y_test, prd, output_dict=True)['accuracy'])

    mean=round(np.mean(partial_accuracies)*100,2)
    deviation=round(np.std(partial_accuracies)*100,2)
    return (str(mean)+"%±"+str(deviation)+"%")

###Line 20 trims 10s data to 1s data. Keep it commented for 10s data. Uncomment for 1s

In [3]:
# Get a list of all files in the current directory
files = [file for file in os.listdir('.') if os.path.isfile(file)]

# Initialize lists to store data
X_list = []
y_list = []

# Loop through each file
for file in files:
    if file.endswith('.json'):
        with open(file, 'r') as f:
            data = json.load(f)
        # Append data to lists
        X_list.append(np.array(data['X'], dtype=np.float32))
        y_list.append(np.array(data['y']))

# Concatenate all datasets
X_combined = np.concatenate(X_list, axis=0)
y_combined = np.concatenate(y_list, axis=0)
#X_combined= X_combined[:, :500] #Keep commented for 10s, uncomment for 1s
print("Combined X shape:", X_combined.shape)
print("Combined y shape:", y_combined.shape)

p_lr= classify_ten_fold_known_noise(X_combined,y_combined)
print ( "Attack Accuracy" + " " + str(p_lr))

Combined X shape: (1400, 5000)
Combined y shape: (1400,)
Attack Accuracy 70.5%±1.84%


#Train a super model and test on FlipStress (Scenario 8a)

In [4]:
# Get a list of all files in the current directory
files = [file for file in os.listdir('.') if os.path.isfile(file)]

# Initialize lists to store data
X_list = []
y_list = []

# Loop through each file
for file in files:
    if file.endswith('.json'):
        with open(file, 'r') as f:
            data = json.load(f)
        # Append data to lists
        X_list.append(np.array(data['X'], dtype=np.float32))
        y_list.append(np.array(data['y']))

# Concatenate all datasets
X_combined = np.concatenate(X_list, axis=0)
y_combined = np.concatenate(y_list, axis=0)

print("Combined X shape:", X_combined.shape)
print("Combined y shape:", y_combined.shape)

X_train=X_combined
y_train=y_combined
mean=X_train.mean()
std=X_train.std()
X_train_norm=(X_train-mean+1e-10)/std

model=LogisticRegression(max_iter=1000)
model.fit(X_train_norm,y_train)

Combined X shape: (1400, 5000)
Combined y shape: (1400,)


LogisticRegression(max_iter=1000)

#Upload the FlipStress dataset at this point

In [6]:
# Function to Test the stressor data
filename = "flipStress_p4_t0.5s.json"
def classify(filename):
  f = open(filename)
  data = json.load(f)
  X_test = np.array(data['X'], dtype=np.float32)
  y_test = np.array(data['y'])

  X_test_norm = (X_test - mean + 1e-10) / std
  prd = model.predict(X_test_norm)

  p_lr=classification_report(y_test, prd, output_dict=True)['accuracy']
  filename = filename[:-5]
  print (filename + " " + str(round(p_lr*100,2)) + "%")


classify(filename)

flipStress_p4_t0.5s 63.5%
